In [ ]:
# thanks to antonoof's work (https://www.kaggle.com/code/antonoof/eda-r-cnn-model), his guide is very good

In [ ]:
# Imports
import os, cv2, json, torch, torchvision, numpy as np, pandas as pd
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torchvision import transforms
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Load and Prepare Data

Let's load all our images into memory for faster training

In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Paths
BASE = '/kaggle/input/recodai-luc-scientific-image-forgery-detection'
authentic_path = f'{BASE}/train_images/authentic'
forged_path = f'{BASE}/train_images/forged'
mask_path = f'{BASE}/train_masks'
test_path = f'{BASE}/test_images'

# Lists to store data
all_images = []
all_labels = []
all_masks = []

print("Loading all images into memory...")

# Load authentic images
for fname in os.listdir(authentic_path):
    if fname.endswith('.jpg') or fname.endswith('.png'):
        img = cv2.imread(os.path.join(authentic_path, fname))
        img = cv2.resize(img, (224, 224))
        all_images.append(img)
        all_labels.append(0)  # 0 = authentic
        all_masks.append(None)

# Load forged images
for fname in os.listdir(forged_path):
    if fname.endswith('.jpg') or fname.endswith('.png'):
        img = cv2.imread(os.path.join(forged_path, fname))
        img = cv2.resize(img, (224, 224))
        all_images.append(img)
        all_labels.append(1)  # 1 = forged

        # Load mask safely
        mask_fname = fname.replace('.jpg', '.npy').replace('.png', '.npy')
        mask_path_full = os.path.join(mask_path, mask_fname)

        mask = np.zeros((224, 224), dtype=np.uint8)  # default mask
        if os.path.exists(mask_path_full):
            loaded_mask = np.load(mask_path_full)
            if loaded_mask is not None and loaded_mask.ndim == 2 and loaded_mask.size > 0:
                mask = cv2.resize(loaded_mask, (224, 224))
        all_masks.append(mask)

print(f"Loaded {len(all_images)} images total")
print(f"Authentic: {all_labels.count(0)}, Forged: {all_labels.count(1)}")

# Custom Dataset
class ForgeryDataset(Dataset):
    def __init__(self, images, labels, masks):
        self.images = images
        self.labels = labels
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img = torch.from_numpy(img).permute(2, 0, 1)  # Channels first

        # Handle masks and bounding boxes
        if label == 1:  # Forged
            mask = self.masks[idx]
            if mask is None:
                mask = np.zeros((224, 224))
            boxes = []
            if mask.max() > 0:
                contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for cnt in contours:
                    x, y, w, h = cv2.boundingRect(cnt)
                    boxes.append([x, y, x+w, y+h])
            if len(boxes) == 0:
                boxes = torch.zeros((0, 4), dtype=torch.float32)
                labels_t = torch.zeros((0,), dtype=torch.int64)
            else:
                boxes = torch.tensor(boxes, dtype=torch.float32)
                labels_t = torch.ones((len(boxes),), dtype=torch.int64)
        else:  # Authentic
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels_t = torch.zeros((0,), dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels_t,
            'image_id': torch.tensor([idx])
        }

        return img, target

# Create dataset instance
dataset = ForgeryDataset(all_images, all_labels, all_masks)

# Function to plot sample images with bounding boxes
def plot_samples(dataset, num_samples=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_samples):
        img, target = dataset[i]
        img_np = img.permute(1, 2, 0).numpy()
        plt.subplot(1, num_samples, i+1)
        plt.imshow(img_np)
        plt.axis('off')

        boxes = target['boxes']
        for box in boxes:
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
            plt.gca().add_patch(rect)
    plt.show()

# Plot 5 sample images
plot_samples(dataset, num_samples=5)


In [ ]:
import matplotlib.pyplot as plt

# Count images per class
authentic_count = all_labels.count(0)
forged_count = all_labels.count(1)

# Bar plot
plt.figure(figsize=(6,4))
plt.bar(['Authentic', 'Forged'], [authentic_count, forged_count], color=['green', 'red'])
plt.title('Number of Images per Class')
plt.ylabel('Count')
plt.show()


## Create Custom Dataset

Simple dataset that returns images and their bounding boxes

In [ ]:
class ForgeryDataset(Dataset):
    def __init__(self, images, labels, masks):
        self.images = images
        self.labels = labels
        self.masks = masks
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        # Convert BGR to RGB (OpenCV loads as BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Normalize to [0, 1] range
        img = img.astype(np.float32) / 255.0
        
        # Convert to tensor (channels first)
        img = torch.from_numpy(img).permute(2, 0, 1)
        
        if label == 1:  # Forged image
            mask = self.masks[idx]
            if mask is None:
                mask = np.zeros((224, 224))
            
            # Find bounding box from mask
            if mask.max() > 0:
                # Find contours
                contours, _ = cv2.findContours(
                    mask.astype(np.uint8), 
                    cv2.RETR_EXTERNAL, 
                    cv2.CHAIN_APPROX_SIMPLE
                )
                
                boxes = []
                for cnt in contours:
                    x, y, w, h = cv2.boundingRect(cnt)
                    boxes.append([x, y, x+w, y+h])
                
                if len(boxes) == 0:
                    boxes = torch.zeros((0, 4), dtype=torch.float32)
                    labels_t = torch.zeros((0,), dtype=torch.int64)
                else:
                    boxes = torch.tensor(boxes, dtype=torch.float32)
                    labels_t = torch.ones((len(boxes),), dtype=torch.int64)
            else:
                boxes = torch.zeros((0, 4), dtype=torch.float32)
                labels_t = torch.zeros((0,), dtype=torch.int64)
        else:  # Authentic
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels_t = torch.zeros((0,), dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'labels': labels_t,
            'image_id': torch.tensor([idx])
        }
        
        return img, target

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

# Create the dataset
dataset = ForgeryDataset(all_images, all_labels, all_masks)

# Function to plot samples with bounding boxes
def plot_samples(dataset, num_samples=5):
    plt.figure(figsize=(15, 5))
    for i in range(num_samples):
        img, target = dataset[i]
        # Convert tensor to numpy image (C,H,W -> H,W,C)
        img_np = img.permute(1, 2, 0).numpy()
        
        plt.subplot(1, num_samples, i+1)
        plt.imshow(img_np)
        plt.axis('off')
        
        boxes = target['boxes']
        # Draw bounding boxes
        for box in boxes:
            x1, y1, x2, y2 = box
            rect = plt.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                fill=False, color='red', linewidth=2
            )
            plt.gca().add_patch(rect)
    plt.show()

# Plot 5 sample images
plot_samples(dataset, num_samples=5)


## Split Data

70/30 split for training and validation

In [ ]:
# Use sklearn for splitting - it's more reliable
train_idx, val_idx = train_test_split(
    range(len(all_images)), 
    test_size=0.3,  # 30% validation
    shuffle=True
)

# Create train/val datasets
train_images = [all_images[i] for i in train_idx]
train_labels = [all_labels[i] for i in train_idx]
train_masks = [all_masks[i] for i in train_idx]

val_images = [all_images[i] for i in val_idx]
val_labels = [all_labels[i] for i in val_idx]
val_masks = [all_masks[i] for i in val_idx]

train_dataset = ForgeryDataset(train_images, train_labels, train_masks)
val_dataset = ForgeryDataset(val_images, val_labels, val_masks)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Build Model

Using Faster R-CNN - it's faster than Mask R-CNN and we can generate masks from bounding boxes!

## Training Setup

Using Adam optimizer - it adapts learning rate automatically which is better than SGD

Forgery_Detector/
│
├─ data/
│   ├─ train/        # images
│   ├─ val/          # images
│   └─ annotations/  # COCO or VOC format annotations
│
├─ models/
│   └─ fasterrcnn_offline.pth  # optional pretrained offline model
│
├─ train.py           # training script
├─ detect.py          # inference & visualization
└─ utils.py           # helper functions (dataset, collate_fn, visualization)


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import VOCDetection
import torchvision.transforms as T
import matplotlib.pyplot as plt
import cv2

# Collate function for detection
def collate_fn(batch):
    return tuple(zip(*batch))

# Simple visualization
def visualize_boxes(image, boxes, labels=None):
    img = image.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.int()
        color = (255,0,0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(img, str(labels[i]), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import cv2

# Collate function for detection datasets
def collate_fn(batch):
    return tuple(zip(*batch))

# Visualize bounding boxes
def visualize_boxes(image, boxes, labels=None):
    img = image.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.int()
        color = (255,0,0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(img, str(labels[i]), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


In [ ]:
# ----- Helper functions (replaces utils.py) -----

import torch
import matplotlib.pyplot as plt
import cv2

# Collate function for object detection
def collate_fn(batch):
    return tuple(zip(*batch))

# Visualize bounding boxes
def visualize_boxes(image, boxes, labels=None):
    img = image.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.int()
        color = (255,0,0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(img, str(labels[i]), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models import resnet18
from torch import nn

# 1️⃣ Create a small ResNet18 backbone (offline)
backbone = resnet18(weights=None)  # no download
backbone = nn.Sequential(*list(backbone.children())[:-2])  # remove last layers
backbone.out_channels = 512  # required by FasterRCNN

# 2️⃣ Create Faster R-CNN model
num_classes = 2  # background + forgery
model = FasterRCNN(backbone, num_classes=num_classes)

# 3️⃣ Move to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print("Offline Faster R-CNN (ResNet18 backbone) ready to use!")


In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, images, targets, transforms=None):
        self.images = images
        self.targets = targets
        self.transforms = transforms

    def __getitem__(self, idx):
        img = self.images[idx]
        target = self.targets[idx]
        if self.transforms:
            img = self.transforms(img)
        return img, target

    def __len__(self):
        return len(self.images)


In [ ]:
import torch
from torch import nn
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# 1️⃣ Create ResNet18 backbone with FPN (fully offline)
backbone = resnet_fpn_backbone('resnet18', pretrained=False)  # no download

# 2️⃣ Create Faster R-CNN model
num_classes = 2  # background + forgery
model = FasterRCNN(backbone, num_classes=num_classes)
model.to(device)

print("Offline Faster R-CNN with ResNet18+FPN ready!")


In [ ]:
import matplotlib.pyplot as plt
import cv2
import torch

def visualize_boxes(image, boxes, labels=None):
    img = image.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        # Convert to Python ints
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        color = (255,0,0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(img, str(labels[i]), (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()


In [ ]:
# Example dummy batch
batch = [(torch.randn(3,224,224), {"boxes": torch.tensor([[10,10,50,50]]), "labels": torch.tensor([1])})]
images, targets = collate_fn(batch)
print(images, targets)

# Example visualization
import numpy as np
dummy_image = np.zeros((100,100,3), dtype=np.uint8)
dummy_boxes = torch.tensor([[10,10,50,50]])
visualize_boxes(dummy_image, dummy_boxes)


In [ ]:
import os

root_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
print(os.listdir(root_dir))  # see what files/folders exist


In [ ]:
import os

train_path = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
print(os.listdir(train_path))  # see all files/folders


In [ ]:
data_loader = DataLoader(
    dataset, 
    batch_size=2, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=4 
)


In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

def __getitem__(self, idx):
    img_path = self.image_paths[idx]
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img)
    target = {
        "boxes": torch.tensor([[0, 0, img.width, img.height]], dtype=torch.float32),
        "labels": torch.tensor([self.labels[idx]], dtype=torch.int64)
    }
    return img_tensor, target


In [ ]:
class ForgeryDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform
        for label, subfolder in enumerate(["authentic", "forged"]):
            folder_path = os.path.join(root_dir, subfolder)
            for f in os.listdir(folder_path):
                if f.endswith((".png", ".jpg", ".jpeg", ".tif")):
                    img = Image.open(os.path.join(folder_path, f)).convert("RGB")
                    self.images.append(img)
                    self.labels.append(label)
        print(f"Loaded {len(self.images)} images into memory")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        if self.transform:
            img = self.transform(img)
        target = {
            "boxes": torch.tensor([[0, 0, img.width, img.height]], dtype=torch.float32),
            "labels": torch.tensor([self.labels[idx]], dtype=torch.int64)
        }
        return img, target


In [ ]:
data_loader = DataLoader(
    dataset, 
    batch_size=2, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True
)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import os

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# -----------------------------
# Collate function
# -----------------------------
def collate_fn(batch):
    return tuple(zip(*batch))

class ForgeryDataset(Dataset):
    def __init__(self, root_dir):
        self.image_paths = []
        self.labels = []  # 0 = authentic, 1 = forged

        # List all images
        for f in os.listdir(root_dir):
            if f.endswith((".png", ".jpg", ".jpeg", ".tif")):
                self.image_paths.append(os.path.join(root_dir, f))
                
                # Example: if filename contains "forged" -> label 1, else 0
                if "forged" in f.lower():
                    self.labels.append(1)
                else:
                    self.labels.append(0)

        print(f"Found {len(self.image_paths)} images")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        img_tensor = torchvision.transforms.functional.to_tensor(img)

        # Placeholder: full image as bounding box
        target = {
            "boxes": torch.tensor([[0, 0, img.width, img.height]], dtype=torch.float32),
            "labels": torch.tensor([self.labels[idx]], dtype=torch.int64)
        }
        return img_tensor, target


In [ ]:
# Adam optimizer is better for this task
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Cosine annealing scheduler - modern approach
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

NUM_EPOCHS = 15
BATCH_SIZE = 1  # Process one image at a time for stability

In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor()
])

data_loader = DataLoader(
    dataset, 
    batch_size=4,        
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,       # parallel loading
    pin_memory=True      # GPU fast transfer
)


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

class ForgeryDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.samples = []
        for label, subfolder in enumerate(["authentic", "forged"]):
            folder = os.path.join(root_dir, subfolder)
            for f in os.listdir(folder):
                if f.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
                    self.samples.append((os.path.join(folder, f), label))
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

dataset = ForgeryDataset("/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images", transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import matplotlib.pyplot as plt

# -----------------------------
# Device
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------------
# Custom Dataset
# -----------------------------
class ForgeryDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.labels = []  # 0 = authentic, 1 = forged
        self.transform = transform

        for label, subfolder in enumerate(["authentic", "forged"]):
            folder_path = os.path.join(root_dir, subfolder)
            for f in os.listdir(folder_path):
                if f.endswith((".png", ".jpg", ".jpeg", ".tif")):
                    self.image_paths.append(os.path.join(folder_path, f))
                    self.labels.append(label)
        print(f"Found {len(self.image_paths)} images")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return img, label


# -----------------------------
# Transforms
# -----------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# -----------------------------
# Dataset & DataLoader
# -----------------------------
train_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
dataset = ForgeryDataset(train_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# -----------------------------
# Offline Model (No Download)
# -----------------------------
model = models.resnet18(pretrained=False)  # offline, no internet
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: authentic/forged
model.to(device)

# -----------------------------
# Loss & Optimizer
# -----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# Training Loop
# -----------------------------
num_epochs = 3
losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(data_loader)
    losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}")

# -----------------------------
# Save Model
# -----------------------------
torch.save(model.state_dict(), "resnet18_forgery_offline.pth")
print("✅ Model saved as resnet18_forgery_offline.pth")

# -----------------------------
# Plot Loss Curve
# -----------------------------
plt.figure(figsize=(6,4))
plt.plot(range(1, num_epochs+1), losses, marker='o')
plt.title("Training Loss Curve (Offline ResNet18)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()


## Helper Function: Box to Mask Conversion

Since Faster R-CNN gives us boxes, we convert them to masks for submission

In [ ]:
def boxes_to_mask(boxes, image_shape):
    """
    Convert bounding boxes to a binary mask
    """
    mask = np.zeros(image_shape, dtype=np.uint8)
    
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Fill the box region
        mask[y1:y2, x1:x2] = 1
    
    return mask

## RLE Encoding

Encode masks as run-length for submission

In [ ]:
def encode_rle(mask):
    """
    Simple RLE encoding
    """
    # Flatten mask row by row
    pixels = mask.flatten()
    
    # Add 0s at start and end for easier calculation
    pixels = np.concatenate([[0], pixels, [0]])
    
    # Find run starts and ends
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    
    # Convert to lengths
    runs[1::2] = runs[1::2] - runs[::2]
    
    # Format as JSON
    result = {
        "counts": runs.tolist(),
        "size": [mask.shape[0], mask.shape[1]]
    }
    
    return json.dumps(result)

## Generate Test Predictions

Process test images and create submission

In [ ]:
import numpy as np
import json

def encode_rle(mask):
    """
    Simple RLE encoding of a binary mask.
    
    Args:
        mask (np.ndarray): 2D binary mask of shape (height, width)
    
    Returns:
        str: JSON string with RLE counts and mask size
    """
    # Flatten mask row by row
    pixels = mask.flatten()
    
    # Add 0s at start and end to detect runs at boundaries
    pixels = np.concatenate([[0], pixels, [0]])
    
    # Find where pixel values change (start/end of runs)
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    
    # Convert starts/ends into lengths
    runs[1::2] = runs[1::2] - runs[::2]
    
    # Format result as JSON
    result = {
        "counts": runs.tolist(),
        "size": [mask.shape[0], mask.shape[1]]
    }
    
    return json.dumps(result)


In [ ]:
MODEL_PATH = "/home/user/projects/models/my_model.h5"


In [ ]:
import os
print(os.getcwd())  # Current working directory
print(os.listdir()) # Files in current directory


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/kaggle/working/resnet18_forgery_offline.pth"

# Load ResNet18
model = models.resnet18(pretrained=False)

# FC layer matches checkpoint (2 outputs)
model.fc = nn.Linear(model.fc.in_features, 2)  # **2 outputs** because checkpoint has 2

# Load checkpoint
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE)
model.eval()

print("Model loaded successfully!")


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# Example data — replace this with your actual predictions
predictions = [0, 1, 0, 1]
ids = [1, 2, 3, 4]

submission = pd.DataFrame({
    'id': ids,
    'label': predictions
})

# Save it to /kaggle/working/
submission.to_csv('/kaggle/working/submission.csv', index=False)

print("✅ Submission file created successfully!")


In [ ]:
import pandas as pd
submission = pd.read_csv('/kaggle/working/submission.csv')
submission.head()
